# Very simple model
This is my jupyter notebook from MESSAGEix workshop. If I double click on this "Markdown" cell, I can edit the text.


In [ ]:
# This is a "Code" cell.
# Importing required packages

import ixmp                    # package for ix modeling platform
import message_ix              # package for MESSAGEix model
from message_ix.utils import make_df

In [ ]:
# Loading the platform (and getting conected to a local database)
mp = ixmp.Platform()
# The path to my local database should be seen after running this cell

In [ ]:
# Building a new, empty MESSAGEix scenario
model = 'coffee_machine'
scenario = 'test'
new = message_ix.Scenario(mp, model, scenario,  version='new')

In [ ]:
# Look for some information
new.version
new?
# new??

# Adding data to the model
## 1) Declaring members of the sets

In [ ]:
new.add_set('technology', ['coffee_maker', 'electricity_grid', 'water_pipe', 'coffee_torrefaction'])
new.add_set('commodity', ['water', 'electr', 'coffee_beans', 'coffee_cup'])       # all members can be added at once
new.add_set('level', ['pipe', 'storage', 'grid', 'useful'])

In [ ]:
# Seeing members of a set
#new.set('commodity')
new.set('technology')

In [ ]:
# Again, like before
new.add_set('node', 'my_home')
new.add_set('year', [2022, 2023])
new.add_set('mode', 'normal')    # modes of operation of a technology

In [ ]:
# listing all the sets (we don't have to add data for all)
new.set_list()

## 2. Adding data to parameters

In [ ]:
new.par_list()

In [ ]:
# List of parameters
new.par_list()
# a python notation
[x for x in new.par_list() if 'emission' in x]

### Adding data to parameter "demand" 
We can always see the indexes of a parameter and make a table with those indexes (plus "value" and "unit"). The method is `idx_names()`


In [ ]:
# Required indexes (sets) for parameter "demand"
new.idx_names("demand")

In [ ]:
# Building the required table (python DataFrame)
import pandas as pd
df = pd.DataFrame({'node': 'my_home',
                   'commodity': 'coffee_cup',
                   'level': 'useful',
                   'year': [2022, 2023],
                   'value': [450, 500],    # cup of coffee per year
                   'unit': '-',
                   'time': 'year'
                   })

In [ ]:
new.add_par('demand', df)
new.par('demand')

In [ ]:
new.idx_names("input")

In [ ]:
new.idx_names('output')

In [ ]:
new.idx_names('var_cost')

## Add technologies for commodities

In [ ]:
new.add_set('technology',['coal_pwr','dark_roaster','municipal_distr_net'])

## Add variable costs for commodities

In [ ]:
costs = {'municipal_distr_net':3,'coal_pwr':20,'dark_roaster':600} # cost of water, electricity, coffee beans
tecs = ['coal_pwr','dark_roaster','municipal_distr_net']
base_df = pd.DataFrame({'node_loc':'my_home','year_vtg':[2022,2023],'year_act':[2022,2023],'mode':'normal','time':'year','unit':'-'})
for tec, value in costs.items():
    df = make_df(base_df,technology=tec,value=value)
    new.add_par('var_cost',df)

In [ ]:
new.par('var_cost')

## Add inputs to coffee machine

In [ ]:
#add water network
df = pd.DataFrame({'node_loc':'my_home', 'technology':'municipal_distr_net','node_origin':'my_home','node_dest':'my_home','mode':'normal',
                   'commodity': 'water','year_vtg':[2022,2023],'year_act': [2022,2023], 
                   'level':'pipe','time':'year','time_origin':'year','time_dest':'year','value':1,'unit':'kg'})
# ['pipe', 'storage', 'grid', 'useful']

new.add_par('output',df)

In [ ]:
# water input to coffee machine
df = pd.DataFrame({'node_loc':'my_home', 'technology':'coffee_maker','node_origin':'my_home','mode':'normal',
                   'commodity': 'water','year_vtg':[2022,2023],'year_act': [2022,2023],
                   'level':'pipe','time':'year','time_origin':'year','value':8,'unit':'kg'})
new.add_par('input',df)

In [ ]:
#add electricity
df = pd.DataFrame({'node_loc':'my_home', 'technology':'coal_pwr','node_origin':'my_home','node_dest':'my_home','mode':'normal',
                   'commodity': 'electr','year_vtg':[2022,2023],'year_act': [2022,2023],
                   'level':'grid','time':'year','time_origin':'year','time_dest':'year','value':1,'unit':'-'})
# ['pipe', 'storage', 'grid', 'useful']

new.add_par('output',df)

In [ ]:
# electricity input to coffee machine

df = pd.DataFrame({'node_loc':'my_home', 'technology':'coal_pwr','node_origin':'my_home','mode':'normal',
                   'commodity': 'electr','year_vtg':[2022,2023],'year_act': [2022,2023],
                   'level':'grid','time':'year','time_origin':'year','value':3,'unit':'-'})

new.add_par('input',df)

In [ ]:
#add beans
df = pd.DataFrame({'node_loc':'my_home', 'technology':'dark_roaster','node_origin':'my_home','mode':'normal',
                   'commodity': 'coffee_beans','year_vtg':[2022,2023],'year_act': [2022,2023],
                   'level':'storage','time':'year','time_origin':'year','value':1,'unit':'kg'})
# ['pipe', 'storage', 'grid', 'useful']

new.add_par('input',df)

In [ ]:
# coffee input to coffee machine
df = pd.DataFrame({'node_loc':'my_home', 'technology':'dark_roaster','node_origin':'my_home','node_dest':'my_home','mode':'normal',
                   'commodity': 'coffee_beans','year_vtg':[2022,2023],'year_act': [2022,2023],
                   'level':'storage','time':'year','time_origin':'year','time_dest':'year','value':5,'unit':'kg'})

new.add_par('output',df)

In [ ]:
new.par('input')

In [ ]:
# Adding data to the parameter "output" for the technology
df = pd.DataFrame({'node_loc': 'my_home',
                   'node_dest': 'my_home',
                   'technology': 'coffee_maker',
                   'commodity': 'coffee_cup',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
new.add_par('output', df)

In [ ]:
# Adding data to the parameter "output" for the technology coffee_torrefaction
df = pd.DataFrame({'node_loc': 'my_home',
                   'node_dest': 'my_home',
                   'technology': 'coffee_torrefaction',
                   'commodity': 'coffee_beans',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
new.add_par('output', df)

In [ ]:
# Adding data to the parameter "output" for the technology electricity_grid
df = pd.DataFrame({'node_loc': 'my_home',
                   'node_dest': 'my_home',
                   'technology': 'electricity_grid',
                   'commodity': 'electr',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
new.add_par('output', df)

In [ ]:
# Adding data to the parameter "output" for the technology water_pipe
df = pd.DataFrame({'node_loc': 'my_home',
                   'node_dest': 'my_home',
                   'technology': 'water_pipe',
                   'commodity': 'water',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 1,
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
new.add_par('output', df)

In [ ]:
# Seeing the content of a parameter
new.par('output')

In [ ]:
new.par_list()

In [ ]:
water_in = pd.DataFrame({'node_loc': 'my_home',
                   'node_origin': 'my_home',
                   'technology': 'coffee_maker',
                   'commodity': 'water',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 10.,
                   'unit': '-',
                   'time': 'year',
                   'time_origin': 'year',
                    })
new.add_par('input', water_in)
new.par('input')

In [ ]:
coffee_beans_in = pd.DataFrame({'node_loc': 'my_home',
                   'node_origin': 'my_home',
                   'technology': 'coffee_maker',
                   'commodity': 'coffee_beans',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 0.03,
                   'unit': '-',
                   'time': 'year',
                   'time_origin': 'year',
                    })
new.add_par('input', coffee_beans_in)
new.par('input')

In [ ]:
elec_in = pd.DataFrame({'node_loc': 'my_home',
                   'node_origin': 'my_home',
                   'technology': 'coffee_maker',
                   'commodity': 'electr',
                   'level': 'useful',
                   'year_vtg': [2022, 2023],   # once installed can be active only one period
                   'year_act': [2022, 2023],
                   'mode': 'normal',
                   'value': 0.1,
                   'unit': '-',
                   'time': 'year',
                   'time_origin': 'year',
                    })
new.add_par('input', elec_in)
new.par('input')

In [ ]:
from message_ix.utils import make_df

base_var_cost = {
    'node_loc': 'my_home',
    'year_vtg': [2022, 2023],
    'year_act': [2022, 2023],
    'mode': 'normal',
    'time': 'year',
    'unit': '-',
}
# in $ / MWh
costs = {
    'coffee_maker': 0.1,
    'electricity_grid': 0.1,
    'water_pipe': 0.01,
    'coffee_torrefaction': 30
}


for tec, val in costs.items():
    df = make_df(base_var_cost, technology=tec, value=val)
    new.add_par('var_cost', df)

In [ ]:
# Saving the data of this scenario
new.commit('I added some data for a coffee maker including costs')   # This message is optional
# If needed to edit after saving the scenario
# new.check_out()                 

In [ ]:
new.version

In [ ]:
# Lets' solve this simple scenario
new.solve()

In [ ]:
# Objective value (why is it zero?)
new.var('OBJ')

In [ ]:
# Activity of coffee maker

# should not have marginals; just levels
new.var('ACT')

In [ ]:
# commodity prices as calculated by the model
new.var('PRICE_COMMODITY')

In [ ]:
# export model parameterization to Excel
new.to_excel("coffee_machine.xlsx")

In [ ]:
# NOTICE: remember to close the platform before openning in another session
mp.close_db()